#### hashMe() function — a helper to create a hash of a message or data

In [3]:
# Importing required libraries
import hashlib
import json
import sys

In [4]:
# Defining a helper function that wraps our hashing algorithm
def hashMe(msg=""):
    if type(msg) != str:
        msg = json.dumps(msg, sort_keys=True)
    
    return hashlib.sha256(str(msg).encode('utf-8')).hexdigest()

In [10]:
transaction = {"sender": "Alice", "recipient": "Bob", "amount": 10}
print(hashMe(transaction))

34bbbd0c0fa96527407730b8eda141d8815d540b464b02dfa0588b0c53382516


#### makeTransaction() funtion — generate exchanges between Alice and Bob

In [ ]:
import random
random.seed(0) # Sets a fixed seed so results are predictable every time you run the code(good for debugging and reproducability)

def makeTransaction(maxValue = 1000): # This will create valid transactions in range of (1, maxValue)
    sign = int(random.getrandbits(1)) * 2 - 1 # This is will randomly choose between -1 or 1 (Picks randomly whether Alice pays Bob or Bob pays Alice)
    amount = random.randint(1, maxValue)
    aliceAmount = sign * amount
    bobAmount = -1 * aliceAmount # Ensures a balanced transaction, and zero-sum

    return {u'Alice': aliceAmount, u'Bob': bobAmount}

In [20]:
for _ in range(5):
    print(makeTransaction())

{'Alice': 224, 'Bob': -224}
{'Alice': 143, 'Bob': -143}
{'Alice': -144, 'Bob': 144}
{'Alice': 98, 'Bob': -98}
{'Alice': 819, 'Bob': -819}


In [21]:
# creating a large set of transactions, then chunking them into blocks
transactionBuffer = [makeTransaction() for _ in range(100)] # 1000 transactions
transactionBuffer

[{'Alice': -932, 'Bob': 932},
 {'Alice': 723, 'Bob': -723},
 {'Alice': 617, 'Bob': -617},
 {'Alice': 151, 'Bob': -151},
 {'Alice': -102, 'Bob': 102},
 {'Alice': 76, 'Bob': -76},
 {'Alice': 871, 'Bob': -871},
 {'Alice': 339, 'Bob': -339},
 {'Alice': -574, 'Bob': 574},
 {'Alice': -363, 'Bob': 363},
 {'Alice': -324, 'Bob': 324},
 {'Alice': 656, 'Bob': -656},
 {'Alice': 210, 'Bob': -210},
 {'Alice': 566, 'Bob': -566},
 {'Alice': -454, 'Bob': 454},
 {'Alice': 534, 'Bob': -534},
 {'Alice': -64, 'Bob': 64},
 {'Alice': 941, 'Bob': -941},
 {'Alice': 938, 'Bob': -938},
 {'Alice': -96, 'Bob': 96},
 {'Alice': 861, 'Bob': -861},
 {'Alice': -728, 'Bob': 728},
 {'Alice': 804, 'Bob': -804},
 {'Alice': 641, 'Bob': -641},
 {'Alice': -627, 'Bob': 627},
 {'Alice': -848, 'Bob': 848},
 {'Alice': 342, 'Bob': -342},
 {'Alice': -748, 'Bob': 748},
 {'Alice': -721, 'Bob': 721},
 {'Alice': 65, 'Bob': -65},
 {'Alice': -940, 'Bob': 940},
 {'Alice': 228, 'Bob': -228},
 {'Alice': -823, 'Bob': 823},
 {'Alice': 146, 'B

#### updateState(transaction, state) funtion — Applying a transaction to account balance
##### This does not validate the transaction, only updates the states. (Only called if the transaction is valid)

In [26]:
def updateState(transaction, state):
    '''
    Inputs: 
        transaction: a dictionary with keys Alice and Bob, and values the amount of money they are sending to each other
        state: a dictionary with keys Alice and Bob, and values their current balance
    Outputs:
        state: a dictionary with keys Alice and Bob, and values their new balance after the transaction
        ** Additional users will be added to state if they are not already present
    '''

    state = state.copy() # Copy the state so we don't modify the original

    for key in transaction.keys():

        if key in state.keys():
            state[key] += transaction[key]
        else:
            state[key] = transaction[key]
    
    return state

In [28]:
state = {'Alice': 1000, 'Bob': 1000} # Initial state of the system
print(f"Initial State:{state}")
transaction = makeTransaction()
print(f"Transaction:{transaction}")

newState = updateState(transaction, state)
print(f"Updated State:{newState}")

Initial State:{'Alice': 1000, 'Bob': 1000}
Transaction:{'Alice': -380, 'Bob': 380}
Updated State:{'Alice': 620, 'Bob': 1380}


#### isValidTransaction(transaction, state) funtion — Validating the transaction

In [29]:
def isValidTransaction(transaction, state):
    '''
    Inputs: 
        transaction: a dictionary with keys Alice and Bob or anyone else, and values the amount of money they are sending to each other
        state: a dictionary with keys Alice and Bob or anyone else, and values their current balance
    Outputs:
        True if the transaction is valid, False otherwise
    '''
    if sum(transaction.values()) != 0:
        return False # Check if the transaction is zero-sum (No new tokens are created or destroyed)
    
    for key in transaction.keys():
        if key in state.keys():
            accountBalance = state[key]
        else:
            accountBalance = 0
        
        if accountBalance + transaction[key] < 0:
            return False
        # Check if the transaction is valid by ensuring that the sender has enough money to send
        # If the sender is not in the state, they are assumed to have 0 balance, so we check if they are sending more than 0

    return True # If all checks pass, the transaction is valid

In [32]:
state = {'Alice': 1000, 'Bob': 1000} # Initial state of the system
print(f"Initial State:{state}")

transaction1 = {'Alice': -800, 'Bob': 800} # Alice sends 800 to Bob
transaction2 = {'Alice': 1500, 'Bob': -1500} # Bob sends 1500 to Alice (invalid transaction since Bob doesn't have enough money)
transaction3 = {'Alice': -400, 'Bob': 600} # Alice sends 400 to Bob but Bob receives 600 (invalid transaction since we can't create or destroy tokens)
transaction4 = {'Alice': -500, 'Bob': 250, 'Lisa': 250} # Creating new users is valid as long as the rules are preserved

print(f"Transaction: {transaction1}, Valid: {isValidTransaction(transaction1,state)}")
print(f"Transaction: {transaction2}, Valid: {isValidTransaction(transaction2,state)}")
print(f"Transaction: {transaction3}, Valid: {isValidTransaction(transaction3,state)}")
print(f"Transaction: {transaction4}, Valid: {isValidTransaction(transaction4,state)}")

Initial State:{'Alice': 1000, 'Bob': 1000}
Transaction: {'Alice': -800, 'Bob': 800}, Valid: True
Transaction: {'Alice': 1500, 'Bob': -1500}, Valid: False
Transaction: {'Alice': -400, 'Bob': 600}, Valid: False
Transaction: {'Alice': -500, 'Bob': 250, 'Lisa': 250}, Valid: True


#### Building the Blockchain: From Transactions to Blocks

In [34]:
state = {'Alice': 100, 'Bob': 100} # Initial state of the system (This is like setting up the original balances in a new blockchain network)

genesisBlockTransactions = state # This is the first block in the blockchain, so we set the transactions to the initial state

genesisBlockContents = {
    'blockNumber': 0,
    'parentHash': None,
    'transactionCount': 1,
    'transactions': genesisBlockTransactions
}

genesisHash = hashMe(genesisBlockContents) # Generating SHA-256 hash of block's content. This makes is immutable. If anything changes, the hash changes

genesisBlock = {'hash': genesisHash, 'contents': genesisBlockContents} # Creating the actual block using a dictionary

genesisBlockStr = json.dumps(genesisBlock, sort_keys=True) # Serializing the block for later use (storing, printing or transmitting)

# This becomes the first element from which everything else will be linked.

In [36]:
# creating the blockchain
chain = [genesisBlock]

#### makeBlock(transaction, chain) function

In [ ]:
def makeBlock(transactions, chain):
    '''
        
    '''
    parentBlock = chain[-1]
    parentHash = parentBlock['hash']
    blockNumber = parentBlock['contents']['blockNumber'] + 1
    transactionCount = len(transactions)

    blockContents = {
        'blockNumber': blockNumber,
        'parentHash': parentHash,
        'transactionCount': transactionCount,
        'transactions': transactions
    }

    blockHash = hashMe(blockContents)

    block = {'hash': blockHash, 'contents': blockContents}

    return block